In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.utils.test_scripts import *
from pkg.planning.pddlstream.plan_rnb import *

In [3]:
rtype = "indy7gripper"
dat_root = "stowing-deep"
res_root = "stowing-deep-result"
dat_dir = "20210829-050034"
file_option = "obj_1"
data_idx = 0
cname = "Full"

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100
MAX_SKELETONS = 10

GRASP_SAMPLE = 100
STABLE_SAMPLE = 100
SEARCH_SAMPLE_RATIO = 10

VISUALIZE = True
SHOW_STATE = True
PLAY_RESULT = True
USE_PYBULLET_GUI = True



CLEARANCE = 1e-3
TOOL_NAME="grip0"
ROBOT_TYPE = {e.name: e for e in RobotType}[rtype]

In [4]:
DATASET_PATH = create_data_dirs(dat_root, rtype, dat_dir)
RESULTSET_PATH = create_data_dirs(res_root, rtype, dat_dir)
print("-"*50)
print("DATASET_PATH: {}".format(DATASET_PATH))
print("RESULTSET_PATH: {}".format(RESULTSET_PATH))
print("-"*50)

--------------------------------------------------
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep/indy7gripper/20210829-050034
RESULTSET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep-result/indy7gripper/20210829-050034
--------------------------------------------------


In [5]:
ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene
crob.home_pose = HOME_POSE
crob.home_dict = list2dict(HOME_POSE, gscene.joint_names)

fname = "data_%s_%02d.pkl" % (file_option, data_idx)
print(fname)
file_gtems = os.path.join(DATASET_PATH, fname)
initial_state = load_saved_scene(pscene, file_gtems, VISUALIZE=VISUALIZE)

gscene.NAME_DICT['obj_0'].color = (1,0,0,1)
gscene.update_markers_all()

mplan = MoveitPlanner(pscene)
checkers = get_checkers_by_case_name(cname, pscene)

mplan.motion_filters = checkers

connection command:
indy0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
data_obj_1_00.pkl


In [7]:
obj_pscene = pscene.subject_dict[pscene.subject_name_list[0]]
obj_pscene.geometry.color = (0.8, 0.2, 0.2, 1)
gscene.update_markers_all()
goal_pairs=[(obj_pscene.oname, 'gp')]

gtimer = GlobalTimer.instance()
gtimer.reset()

res, plan, log_dict = solve_in_pddlstream(pscene, mplan, ROBOT_NAME, TOOL_NAME, HOME_POSE, goal_pairs, 
                        TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS, 
                        GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO, 
                        use_pybullet_gui=USE_PYBULLET_GUI)

# save_pickle(os.path.join(RESULTSET_PATH, "result_%s_%02d_%s.pkl" % (file_option, data_idx, cname)), log_dict)

print("------- Result {} ({}): {} s -------".format(fname, cname, log_dict["plan_time"]))
print("==========================================================")
print("==========================================================")
print(gtimer)
print("==========================================================")
print("==========================================================")


[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
Objects: {1L: 'obj_0', 2L: 'gp', 3L: 'obj_2', 4L: 'obj_1', 5L: 'floor', 6L: 'base', 7L: 'wp', 8L: 'obs_0'}
================ MAKE PROBLEM ======================
IK checkers: ['GraspChecker', 'ReachChecker']
MP checkers: ['GraspChecker', 'ReachChecker', 'LatticedChecker']
timeout motion : 5
Robot: 0
Movable: [3L, 4L, 1L]
Fixed: [2, 5, 6, 7, 8]
body 3 - surface 2
body 3 - surface 5
body 3 - surface 6
body 3 - surface 7
body 3 - surface 8
body 4 - surface 2
body 4 - surface 5
body 4 - surface 6
body 4 - surface 7
body 4 - surface 8
body 1 - surface 2
body 1 - surface 5
body 1 - surface 6
body 1 - surface 7
body 1 - surface 8
Init: [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Graspable', 3L), ('Pose', 3L, p0), ('AtPose', 3L, p0), ('Stackable', 3L, 2), ('Stackable', 3L, 5), ('Stackable', 3L, 6), ('Stackable', 3L, 7), ('Stackable', 3L, 8), ('Graspable', 4L), ('Pose', 4L, p1), ('A

iter=0, outs=1) sample-grasp:(4)->[(g3)]
Check Feas Fail: GraspChecker
iter=inf, outs=0) inverse-kinematics:(4, p1, g3)->[]
iter=1, outs=1) sample-grasp:(4)->[(g4)]
IK-col approach fail
IK-col approach fail
IK-col approach fail
IK-col approach fail
IK-col approach fail
IK-col approach fail
IK-col approach fail
inverse_kinematics fail to approach
IK-col approach fail
inverse_kinematics fail to approach
IK-col approach fail
inverse_kinematics fail to approach
IK-col approach fail
iter=inf, outs=0) inverse-kinematics:(4, p1, g4)->[]
iter=2, outs=1) sample-grasp:(4)->[(g5)]
Check Feas Fail: GraspChecker
iter=inf, outs=0) inverse-kinematics:(4, p1, g5)->[]

Iteration: 7 | Complexity: 4 | Skeletons: 2 | Skeleton Queue: 5 | Disabled: 0 | Evaluations: 59 | Eager Calls: 0 | Cost: inf | Search Time: 0.904 | Sample Time: 21.698 | Total Time: 22.602
Attempt: 1 | Results: 160 | Depth: 1 | Success: False | Time: 0.264
Attempt: 2 | Results: 321 | Depth: 0 | Success: True | Time: 0.526
Stream plan (28

Attempt: 1 | Results: 538 | Depth: 0 | Success: True | Time: 0.478
Stream plan (38, 14, 0.001): [sample-grasp:(1)->(g2), inverse-kinematics:(1, p2, g2)->(q1, c0), test-cfree-traj-pose:(c0, 1, p2)->(), sample-pose:(1, 2)->(p5), inverse-kinematics:(1, p5, g2)->(q2, c1), test-cfree-traj-pose:(c1, 3, p0)->(), test-cfree-traj-pose:(c1, 4, p1)->(), sample-grasp:(3)->(g6), inverse-kinematics:(3, p0, g6)->(q4, c4), test-cfree-traj-pose:(c4, 1, p2)->(), test-cfree-traj-pose:(c0, 4, p1)->(), test-cfree-traj-pose:(c4, 4, p1)->(), test-cfree-traj-pose:(c0, 3, p0)->(), test-cfree-traj-pose:(c4, 3, p0)->(), sample-pose:(1, 6)->(#p5), inverse-kinematics:(1, #p5, g2)->(#q8, #t956), test-cfree-traj-pose:(#t956, 3, p0)->(), test-cfree-traj-pose:(#t956, 4, p1)->(), test-cfree-traj-pose:(#t956, 1, #p5)->(), test-cfree-pose-pose:(3, p0, 1, p2)->(), test-cfree-pose-pose:(1, p5, 4, p1)->(), test-cfree-pose-pose:(1, p5, 3, p0)->(), test-cfree-pose-pose:(3, p0, 4, p1)->(), test-cfree-pose-pose:(1, #p5, 3, p0)-

iter=inf, outs=0) inverse-kinematics:(1, p10, g2)->[]
IK-col approach fail
IK-col approach fail
inverse_kinematics fail to approach
iter=inf, outs=1) inverse-kinematics:(1, p6, g0)->[(q8, c11)]
iter=inf, outs=1) test-cfree-traj-pose:(c11, 4, p1)->[()]
iter=inf, outs=1) test-cfree-traj-pose:(c11, 3, p0)->[()]
sucject/actor: obj_0 / wp
Check Feas Fail: GraspChecker
holding motion: False - True

         20865284 function calls (20496520 primitive calls) in 130.075 seconds

   Ordered by: internal time
   List reduced from 1767 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        6   69.021   11.504   69.021   11.504 /usr/local/lib/python2.7/dist-packages/zmq/sugar/socket.py:465(recv_multipart)
    20077   27.375    0.001   27.375    0.001 {time.sleep}
        7   16.832    2.405   16.835    2.405 pkg/planning/motion/moveit/moveit_py.py:158(plan_joint_motion_py)
    13203    1.461    0.000    2.016    0.000 /usr/local/lib/python2.7

KeyboardInterrupt: 

In [8]:
if VISUALIZE and PLAY_RESULT and res:
    play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
                   body_names=log_dict['body_names'], plan=plan, SHOW_PERIOD=0.01)


## Old version

In [6]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [7]:
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

In [10]:
from_state = initial_state.copy(pscene)
from_state.Q = HOME_POSE
goal_nodes = [('gp',)]
ppline.search(from_state, goal_nodes, verbose=True, display=False, 
              timeout_loop=100, multiprocess=False, timeout=1, add_homing=False)

try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: GraspChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
Motion Filer Failure: LatticedChecker
result: 0 - ('wp',)->('grip0',) = fail
try: 0 - ('wp',)->('grip0',)
try transition motion
transition motion tried: True
result: 0 - ('wp',)->('grip0',) = success
branching: 0->1 (0.19/100.0 s, steps/err: 52(108.756065369 ms)/0.00122731377613)
try: 1 - ('grip0',)->('gp',)
try transition motion
transition motion tried: True
Goal reached
result: 1 - ('grip0',)->('gp',) = success
branching: 1->2 (0.45/100.0 s, steps/err: 77(266.849994659 ms)/0.00176797387511)
======================= terminated 0: first answer acquired ===============================


In [11]:
snode_schedule = ppline.tplan.get_best_schedule()

In [12]:
ppline.play_schedule(snode_schedule)

('wp',)->('grip0',)
('grip0',)->('gp',)


In [10]:
pscene.set_object_state(initial_state)

In [15]:
if VISUALIZE and PLAY_RESULT and res:
    play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
                   body_names=body_names, plan=plan, SHOW_PERIOD=0.01)

s_builder.xcustom.clear()